In [1]:
import subprocess
import os
import sys
sys.path.append('../')
import utils as ul
import copy
import pickle
import gzip
from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import medfilt
import h5py
from scipy.stats import norm
import time
import pymultinest
import json
import copy

in_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/'
lya_hits = pickle.load(gzip.open(in_dir+'lya_hits_cut.pzip'))

dir1 = os.getcwd()
os.chdir('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/2')
dir2 = os.getcwd()

plate_list = lya_hits['plate_hits']
fiber_list = lya_hits['fiber_hits']
mjd_list = lya_hits['mjd_hits']
spec_id_list = lya_hits['spec_id']
peak_id_list = lya_hits['peak_id']
peak_wave_list = lya_hits['peak_wave']
peak_sn_list = lya_hits['peak_sn']

t1 = time.time()
for i in range(plate_list.size):
    this_plate = plate_list[i]
    this_mjd = mjd_list[i]
    this_fiber = fiber_list[i]
    this_spec_id = spec_id_list[i]
    this_peak_id = peak_id_list[i]
    this_peak_wave = peak_wave_list[i]
    this_peak_sn = peak_sn_list[i]
    h5_file = '{}_{}.h5'.format(this_plate,this_mjd)
    with h5py.File(h5_file, 'r') as f:
        arr0 = f['flux']
        flux = arr0[:]
        
        arr1 = f['loglam']
        loglam = arr1[:]
        
        arr2 = f['zans']
        zans = arr2[:]
        
        arr3 = f['newsynflux']
        synflux = arr3[:]
        
        arr4 = f['res_ivar']
        res_ivar = arr4[:]
    
    
    this_wave = 10**loglam
    this_flux = flux[this_spec_id,:]
    this_synflux = synflux[this_spec_id,:]
    this_resflux = this_flux - this_synflux
    this_res_ivar = res_ivar[this_spec_id,:]
    this_sn = np.sqrt(this_resflux**2 * this_res_ivar)

    
    hw_sub = 50 #empirically choice
    wave_sub = this_wave[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    resflux_sub =  this_resflux[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    res_ivar_sub = this_res_ivar[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    sn_sub = this_sn[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    
    #------------------------------fitting block---------------
    #fitting x-coordinates
    npix = wave_sub.size
    wave_fit = np.arange(0,npix)
    #fitting data/uncertainty
    dlam = wave_sub * 1.0e-4 * np.log(10.0)
    obs = np.copy(resflux_sub * dlam)
    weight = np.copy(res_ivar_sub / dlam**2)
    ul.npix = np.copy(npix)
    ul.obs = np.copy(obs)
    ul.weight = np.copy(weight)
    ul.wave_fit = np.copy(wave_fit)
    
    png_name = '{}-{}-{}'.format(this_plate,this_mjd,this_fiber)
    multinest_base_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/'
   
    #single peak
    if not os.path.exists(multinest_base_dir+'single/'+png_name+'_1/'):  
        multinest_output = multinest_base_dir+'single/'+png_name+'_1/'
        os.mkdir(multinest_output)
    else:
        output = subprocess.Popen('ls -d {}single/{}*'.format(multinest_base_dir, png_name),stdout=subprocess.PIPE,shell=True)
        #print('ls -d {}single/{}*'.format(multinest_base_dir, out_png))
        lines = output.stdout.read().splitlines()
        #print(lines,'>>>>')
        k_hits = len(lines)
        multinest_output = multinest_base_dir+'single/'+png_name+'_{}/'.format(k_hits+1)
        os.mkdir(multinest_output)
    parameters = ['A', 'x', 'omega', 's']
    n_params = len(parameters)
    pymultinest.run(ul.loglike_single, ul.prior_single, n_params, resume =False, verbose = True,outputfiles_basename=multinest_output,
                   n_live_points=300, evidence_tolerance=0.5,sampling_efficiency=0.05,
                   importance_nested_sampling=False,const_efficiency_mode=True, multimodal=True)
    json.dump(parameters, open(multinest_output+'params.json', 'w'))
    res = pymultinest.Analyzer(outputfiles_basename=multinest_output, n_params = n_params)
    xx = res.get_best_fit()
    #print(xx)
    like_single = xx['log_likelihood']
    ev_single = res.get_stats()['global evidence']
    print('evidence value:',ev_single)
    fit_par_single = xx['parameters']
    #plt.figure()
    #plt.bar(wave_fit,obs,alpha=0.5,width=1)
    #plt.plot(wave_fit,ul.skew_gauss(npix , *fit_par,hw_sub=10),color='red')
    #plt.show()
    
    #double peak
    if not os.path.exists(multinest_base_dir+'double/'+png_name+'_1/'):  
        multinest_output = multinest_base_dir+'double/'+png_name+'_1/'
        os.mkdir(multinest_output)
    else:
        output = subprocess.Popen('ls -d {}double/{}*'.format(multinest_base_dir, png_name),stdout=subprocess.PIPE,shell=True)
        lines = output.stdout.read().splitlines()
        k_hits = len(lines)
        multinest_output = multinest_base_dir+'double/'+png_name+'_{}/'.format(k_hits+1)
        os.mkdir(multinest_output)
    parameters = ['flux_ratio', 'offset', 'omega1', 's1', 'A2', 'x2', 'omega2', 's2']
    n_params = len(parameters)
    pymultinest.run(ul.loglike_double, ul.prior_double, n_params, resume =False, verbose = True,outputfiles_basename=multinest_output,
                   n_live_points=300, evidence_tolerance=0.5,sampling_efficiency=0.05,
                   importance_nested_sampling=False,const_efficiency_mode=True, multimodal=True)
    json.dump(parameters, open(multinest_output+'params.json', 'w'))
    res = pymultinest.Analyzer(outputfiles_basename=multinest_output, n_params = n_params)
    xx = res.get_best_fit()
    #print(xx)
    like_double = xx['log_likelihood']
    ev_double = res.get_stats()['global evidence']
    print('evidence value:',ev_double)
    fit_par_double = xx['parameters']
    #plt.figure()
    #plt.bar(wave_fit,obs,alpha=0.5,width=1)
    #plt.plot(wave_fit,ul.double_peak(npix , *fit_par,hw_sub=10),color='red')
    #plt.show()
    #------------------------------fitting block---------------
    
    #make sure reading process is correct
    #print(zans[this_spec_id])
    #print(this_plate,this_mjd,this_fiber)
    
    #visulization
    import matplotlib.gridspec as gridspec
    name = r'ID: {}-{}-{}'.format(this_plate,this_mjd,this_fiber)
    hit_wave = r'Hit $\lambda$: {:.2f}'.format(this_peak_wave)
    hit_sn = r'Hit SN: {:.2f}'.format(this_peak_sn)
    textstr = '\n'.join((name,hit_wave,hit_sn))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    
    #fig, ax = plt.subplots(2,2,figsize=(20,15)) #,sharey=True
    fig = plt.figure(figsize=(50,30),tight_layout=True)
    gs = gridspec.GridSpec(3, 5)
    
    #--------------------------subplot 0,0:
    ax = fig.add_subplot(gs[0, :])
    ax.plot(this_wave,this_flux,color='deepskyblue')
    ax.plot(this_wave, this_synflux,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    id_mask = ul.return_mask_region_id(this_res_ivar)
    for this_mask_id in id_mask:
        low = this_mask_id[0]
        high = this_mask_id[1]
        #print(this_wave[low],this_wave[high-1])
        ax.axvspan(this_wave[low], this_wave[high-1], facecolor='g', alpha=0.5)
    #ax[0,0].set_xlim(3700,4800)
    up_lim = this_flux.max()*1.2
    if this_flux.min() < 0:
        low_lim = this_flux.min()*1.2
    else:
        low_lim = -0.1
    ax.set_ylim(low_lim,up_lim)
    
    
    #--------------------------subplot 1,1
    ax = fig.add_subplot(gs[1, 1:])
    ax.plot(this_wave,this_flux,color='deepskyblue')
    ax.plot(this_wave, this_synflux,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    id_mask = ul.return_mask_region_id(this_res_ivar)
    for this_mask_id in id_mask:
        low = this_mask_id[0]
        high = this_mask_id[1]
        #print(this_wave[low],this_wave[high-1])
        ax.axvspan(this_wave[low], this_wave[high-1], facecolor='g', alpha=0.5)
    ax.set_xlim(3700,4800)
    id_tmp = np.where(np.logical_and(this_wave<4800,this_wave>3700))
    up_lim = this_flux[id_tmp].max()*1.2
    if this_flux[id_tmp].min() < 0:
        low_lim = this_flux[id_tmp].min()*1.2
    else:
        low_lim = -0.1
    ax.set_ylim(low_lim,up_lim)
    
    
    #--------------------------subplot 2,0
    ax = fig.add_subplot(gs[2, 0])
    ax.bar(wave_sub,sn_sub,alpha=0.5)
    ax.plot(wave_sub,sn_sub,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    wave_, sn_ = ul.return_peak(wave_sub,sn_sub)
    ax.plot(wave_,sn_,'*',color='black',ms=20)
    
    
    #--------------------------subplot 2,1
    ax = fig.add_subplot(gs[2, 1])
    _,sn_filter_0,_ = ul.filter_sn(resflux=resflux_sub,resivar=res_ivar_sub,wave=wave_sub,pix_sigma=1.1,kern_size=15)
    ax.bar(wave_sub,sn_filter_0,alpha=0.5)
    ax.plot(wave_sub,sn_filter_0,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    ax.text(0.05, 0.95, 'filtered spectrum 1.1', transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    wave_, sn_ = ul.return_peak(wave_sub,sn_filter_0)
    ax.plot(wave_, sn_,'*',color='black',ms=20)
    
    
    #--------------------------subplot 2,2
    ax = fig.add_subplot(gs[2, 2])
    _,sn_filter_1,_ = ul.filter_sn(resflux=resflux_sub,resivar=res_ivar_sub,wave=wave_sub,pix_sigma=2.2,kern_size=15)
    ax.bar(wave_sub,sn_filter_1,alpha=0.5)
    ax.plot(wave_sub,sn_filter_1,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    ax.text(0.05, 0.95, 'filtered spectrum 2.2', transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    wave_, sn_ = ul.return_peak(wave_sub,sn_filter_1)
    ax.plot(wave_, sn_,'*',color='black',ms=20)
    
    #--------------------------subplot 2,3
    ax = fig.add_subplot(gs[2, 3])
    ax.bar(wave_fit,obs,alpha=0.5,width=1)
    ax.plot(wave_fit,ul.skew_gauss(npix , *fit_par_single,hw_sub=2),color='red')
    ax.text(0.05, 0.95, 'Ev: {}\nLike: {}'.format(ev_single,like_single), transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    

    #--------------------------subplot 2,4
    ax = fig.add_subplot(gs[2, 4])
    ax.bar(wave_fit,obs,alpha=0.5,width=1)
    ax.plot(wave_fit,ul.double_peak(npix , *fit_par_double,hw_sub=2),color='red')
    ax.text(0.05, 0.95, 'Ev: {}\nLike: {}'.format(ev_double,like_double), transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    
    #--------------------------subplot 1,0
    data_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/cross_talk/'
    color = ['black','red','cyan']
    ax = fig.add_subplot(gs[1, 0])
    for jj in range(3):
        h5_file = data_dir + '{}_{}_{}_{}.h5'.format(this_plate,this_mjd, this_fiber,jj)
        with h5py.File(h5_file, 'r') as f:
            arr0 = f['flux']
            flux = arr0[:]
            
            arr1 = f['wave']
            wave = arr1[:]
            
            arr2 = f['zans']
            zans = arr2[:]
        ax.plot(wave,flux,color=color[jj])
        ax.set_xlim(this_peak_wave-50,this_peak_wave+50)
        ax.minorticks_on()
    
    #plt.show()
    #png_name = '{}-{}-{}'.format(this_plate,this_mjd,this_fiber)
    out_png = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/{}'.format(png_name)
    if not os.path.exists(out_png+'_1.png'):  
        fig.savefig(out_png+'_1.png',dpi=40,bbox_inches='tight')
        plt.close(fig)
        print(out_png+'_1.png','---{} image-----finished!'.format(i))
    else:
        output = subprocess.Popen('ls -d {}*'.format(out_png),stdout=subprocess.PIPE,shell=True)
        lines = output.stdout.read().splitlines()
        print('----lines:',lines)
        k_hits = len(lines)
        fig.savefig(out_png+'_{}.png'.format(k_hits+1),dpi=50,bbox_inches='tight')
        plt.close(fig)
        print(out_png+'_{}.png'.format(k_hits+1),'---{} image-----finished!'.format(i),'!!!!!!')
t2 = time.time()
print('total time elapse:',t2-t1)

  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/single/10243-58159-381_1/.txt
('evidence value:', -56.36499079665449)
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/double/10243-58159-381_1/.txt
('evidence value:', -61.64603285107123)


/data/inspur_disk03/userdir/caoxy/software/anaconda2/lib/python2.7/site-packages/matplotlib/figure.py:2299: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10243-58159-381_1.png', '---0 image-----finished!')
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/single/10272-58172-937_1/.txt
('evidence value:', -73.27241611161709)
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/double/10272-58172-937_1/.txt
('evidence value:', -75.23597680812485)
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10272-58172-937_1.png', '---1 image-----finished!')
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/single/10274-58170-211_1/.txt
('evidence value:', -49.34726255378325)
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/double/10274-58170-211_1/.txt
('evidence value:', -52.49926446802945)
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10274-58170-211_1.png', '---2 image-----finished!')
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya

../utils.py:305: RuntimeWarning: divide by zero encountered in divide
  gauss_comp2 = np.exp(-0.5*np.power((lam-x2)/omega2,2))
../utils.py:306: RuntimeWarning: divide by zero encountered in divide
  erf_comp2 = factor * ( erf(s2*(lam-x2)/omega2)+1.0 )


  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/double/4667-55868-155_1/.txt
('evidence value:', -56.222293343145346)
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/4667-55868-155_1.png', '---453 image-----finished!')
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/single/4686-56013-701_1/.txt
('evidence value:', -32.38913264479778)
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/double/4686-56013-701_1/.txt
('evidence value:', -34.04559347051702)
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/4686-56013-701_1.png', '---454 image-----finished!')
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/single/4687-56369-25_1/.txt
('evidence value:', -62.97044655441052)
  analysing data from /data/inspur_disk03/userdir/caoxy/eboss_lya/data/multinest_out/double/4687-56369-25_1/.txt
('evidence value:', -54.95618667956144)
('/data/inspur_disk

In [1]:
pwd

u'/data/inspur_disk03/userdir/caoxy/eboss_lya/py/visidual_inspec_module'

In [2]:
import copy
import pickle
import gzip
from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import medfilt
import h5py

in_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/'
lya_hits = pickle.load(gzip.open(in_dir+'lya_hits_cut.pzip'))


In [5]:
plate_list

array([10243, 10272, 10274, ...,  9631,  9631,  9632])

# Note
this work should use the results from cross talk fie(just run all it procudure in notebook file)

In [19]:
xx['log_likelihood']

-42.592241708825235

In [2]:
83366/3600

23

In [1]:
import subprocess
import os
import sys
sys.path.append('../')
import utils as ul
import copy
import pickle
import gzip
from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import medfilt
import h5py
from scipy.stats import norm
import time
import pymultinest
import json
import copy

in_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/'
lya_hits = pickle.load(gzip.open(in_dir+'lya_hits_cut.pzip'))

dir1 = os.getcwd()
os.chdir('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/2')
dir2 = os.getcwd()

plate_list = lya_hits['plate_hits']
fiber_list = lya_hits['fiber_hits']
mjd_list = lya_hits['mjd_hits']
spec_id_list = lya_hits['spec_id']
peak_id_list = lya_hits['peak_id']
peak_wave_list = lya_hits['peak_wave']
peak_sn_list = lya_hits['peak_sn']

In [4]:
np.where(plate_list==8501)

(array([], dtype=int64),)